# Exploratory Analysis Phishing_Email.csv Group 3

In [ ]:
# Potentionally helpful libraries:
# pandas, matplotlib, seaborn, nltk, scikit-learn, gensim, pyLDAvis

import pandas as pd
import re

In [ ]:
# Reading in the data
df_phish = pd.read_csv("Phishing_Email.csv")
# Dropping the unecessary index row
df_phish = df_phish.drop(df_phish.columns[0], axis=1)
#checking that it read in correctly
df_phish[0:10]

In [ ]:
# Pulling out a few Emails as examples
#for i in range(2):
#    print("\n\nBEGIN NEW EMAIL:")
#    print(df_phish.loc[i+22,"Email Text"])




## Data Cleaning

In [ ]:
NullCount = df_phish["Email Text"].isnull().sum()
print("Nulls:", NullCount)
# Check for null values in the "Email Text" column
null_mask = df_phish["Email Text"].isnull()

# If there are null values, drop the corresponding rows
if null_mask.any():
    df_phish = df_phish.drop(df_phish[null_mask].index)
# Checking if null values were dropped
NullCount = df_phish["Email Text"].isnull().sum()
print("Nulls:", NullCount)

# Visual Inspection
print(df_phish.head(10))
print(df_phish.tail(20))


In [ ]:
# Check if each "Email Text" is type string
print(df_phish.shape)
# Check if each "Email Text" is type string
is_string = df_phish["Email Text"].apply(lambda x: isinstance(x, str))
print("Number of non-string values in 'Email Text':", (~is_string).sum())
print(df_phish.shape)

In [ ]:
#Get count of 'empty' emails
empty_email_count = df_phish['Email Text'].str.lower().eq('empty').sum()
print(f"Number of 'empty' emails: {empty_email_count}")
#Remove those 'empty' emails from the dataframe
#df_phish_cleaned = df_phish[df_phish['Email Text'].str.lower() != 'empty']
#df_phish_cleaned

## Feature Extraction

### Emails with "re:"

In [ ]:
print(df_phish.tail(5))
countNAN = df_phish["Email Text"] == "NaN"
print(countNAN.sum())
# Adding "Is_Response" feature
df_phish["Is_Response"] = None
# Looping over the DataFrame to determine "Is_Response"
for i in range(df_phish.shape[0]):
    has_re = re.search('re :', df_phish.iloc[i]["Email Text"])
    df_phish.at[i, "Is_Response"] = bool(has_re)
print(df_phish.tail(20))

In [ ]:
df_phish["Is_Response"].value_counts(normalize=True)

### Emails with links

In [ ]:
print(df_phish.tail(5))
# Adding "Has_WebLink" and "WebLink" column
df_phish["Has_WebLink"] = None
df_phish["WebLink"] = None
# Patterns to search for hyperlinks, This may need updating to include more patterns
patterns = [r'https?://\S+', r'www\.\S+']

# Looping over the DataFrame
for i in range(df_phish.shape[0]):
    has_link = False
    link = None
    # Looping over each pattern
    for pattern in patterns:
        match = re.search(pattern, str(df_phish.iloc[i]["Email Text"]))
        if match:
            has_link = True
            # write something here to record the link
            link = match.group()
            break  # Break the loop if any pattern is found
    # Update the "Has_WebLink" column based on whether a link is found
    df_phish.at[i, "Has_WebLink"] = has_link
    df_phish.at[i, "WebLink"] = link
print(df_phish.tail(5))
    

In [ ]:
df_phish["Has_WebLink"].value_counts(normalize=True)

### Email Length

In [ ]:
# Creating the Email Length Column
df_phish["Email_Length"] = None
for i in range(df_phish.shape[0]):
    df_phish.at[i, "Email_Length"] = len(str(df_phish.iloc[i]["Email Text"]))
    
# Verifying it worked
print(df_phish["Email_Length"].tail(10))
print(df_phish.tail(5))
